### **Configuration for Data Quality and Quarantine Framework**

In [0]:
#%sql
#CREATE CATALOG IF NOT EXISTS main;

In [0]:
#just checking the changes is being deployed from the vscode to databricks or not

In [0]:
%sql
USE CATALOG main;
CREATE SCHEMA IF NOT EXISTS main.meta;


In [0]:
 %sql
CREATE TABLE main.meta.enrollment as Select * from healthverity_claims_sample_patient_dataset.hv_claims_sample.enrollment

In [0]:
%sql
CREATE TABLE main.meta.pharmacy_claim as Select * from healthverity_claims_sample_patient_dataset.hv_claims_sample.pharmacy_claim;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS main.meta.rule_defination (
  rule_id     INT         PRIMARY KEY,    
  rule_type   STRING      NOT NULL,  
  threshold   STRING,  
  active_flag BOOLEAN     DEFAULT TRUE,
  description STRING
)
USING DELTA
TBLPROPERTIES ('delta.feature.allowColumnDefaults'='supported');


In [0]:
%sql
CREATE TABLE IF NOT EXISTS main.meta.column_map (
  table_name   STRING      NOT NULL,      
  column_name  STRING      NOT NULL,     
  rule_ids     STRING      NOT NULL,   
  created_ts  TIMESTAMP
)
USING DELTA
TBLPROPERTIES ('delta.feature.allowColumnDefaults'='supported');


In [0]:
%sql
CREATE TABLE IF NOT EXISTS main.meta.bronze_dq_quarantine (
    run_id BIGINT,                     -- Pipeline execution ID
    source_table STRING,               -- Logical source table name
    _row_id BIGINT,                    -- Internal row identifier (monotonic ID)
    invalid_data STRING,               -- JSON snapshot of the invalid row
    failed_rule_id INT,                -- Rule ID
    failed_rule_type STRING,           -- Rule type (null_check, regex, range, duplicate)
    failure_reason STRING,             -- Human-readable explanation

    processed_timestamp TIMESTAMP      -- When record was quarantined
)
USING DELTA;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS main.meta.dq_column_group (
    table_name      STRING   COMMENT 'Logical table name (without bronze_)',
    column_name     STRING   COMMENT 'Column to be aggregated',
    merge_strategy  STRING   COMMENT 'Aggregation strategy: concat | max | min | first',
    delimiter       STRING   COMMENT 'Delimiter for concat strategy (nullable)',
    active_flag     STRING   COMMENT 'Y/N to enable or disable aggregation',
    created_ts      TIMESTAMP DEFAULT current_timestamp(),
    updated_ts      TIMESTAMP
)
USING DELTA
TBLPROPERTIES ('delta.feature.allowColumnDefaults'='supported');


In [0]:
%sql
CREATE TABLE IF NOT EXISTS main.meta.dq_observability_summary (
    run_id           BIGINT,
    source_table     STRING,
    metric_ts        TIMESTAMP,   
    metric_date      DATE,        
    metric_category  STRING,
    metric_name      STRING,
    metric_value     STRING,
    created_ts       TIMESTAMP
)
USING DELTA
PARTITIONED BY (metric_date);

In [0]:
%sql
-- ─────────────────────────────────────────────
-- RULE DEFINITIONS (Enrollment + Pharmacy Claim)
-- ─────────────────────────────────────────────

INSERT OVERWRITE main.meta.rule_defination
(rule_id, rule_type, threshold, active_flag, description)
VALUES
  (1, 'null_check', 'NA', TRUE, 'Checks for NULL values'),

  (2, 'duplication', 'KEY_BASED', TRUE,
   'Checks for duplicate values based on configured key columns'),

  (3, 'range_dob', '1900-2025', TRUE,
   'Checks if value falls within a reasonable range (e.g., year of birth)'),

  (4, 'regex_gender', '^[MF]$', TRUE,
   'Checks if value is M or F (for gender)'),

  (5, 'regex_num', '^\\d{3}$', TRUE,
   'Checks if value is a 3-digit number (for zip3)'),

  (6, 'regex_state', '^[A-Z]{2}$', TRUE,
   'Checks if value is two uppercase letters (for state)'),

  (7, 'schema_validation', 'int', TRUE,
   'Checks if value is an integer (for year of birth)'),

  -- Pharmacy Claim Specific Rules
  (8, 'range_momentary', '0-999999', TRUE,
   'Checks if monetary values fall within a reasonable range'),

  (9, 'regex_uppercase', '^[A-Z0-9]+$', TRUE,
   'Checks if NDC code is alphanumeric and upper case'),

  (10, 'regex_date', '^\\d{4}-\\d{2}-\\d{2}$', TRUE,
   'Checks if date_service is in YYYY-MM-DD format');


In [0]:
%sql
INSERT OVERWRITE main.meta.column_map
SELECT * FROM VALUES
-- ENROLLMENT
('enrollment','patient_id','1,2',current_timestamp()),
('enrollment','date_start','1,10,2',current_timestamp()),
('enrollment','date_end','1,10,2',current_timestamp()),
('enrollment','patient_gender','1,4',current_timestamp()),
('enrollment','patient_year_of_birth','1,3,7',current_timestamp()),
('enrollment','patient_zip3','1,5',current_timestamp()),
('enrollment','patient_state','1,6',current_timestamp()),
('enrollment','benefit_type','1,2',current_timestamp()),
('enrollment','pay_type','1',current_timestamp()),

-- PHARMACY_CLAIM
('pharmacy_claim','claim_id','1,2',current_timestamp()),
('pharmacy_claim','fill_number','1,2',current_timestamp()),
('pharmacy_claim','patient_id','1,2',current_timestamp()),
('pharmacy_claim','ndc','1,9,2',current_timestamp()),
('pharmacy_claim','date_service','1,10,2',current_timestamp()),
('pharmacy_claim','submitted_gross_due','1,8,2',current_timestamp()),
('pharmacy_claim','paid_gross_due','1,8,2',current_timestamp()),
('pharmacy_claim','copay_coinsurance','1,8,2',current_timestamp()),
('pharmacy_claim','days_supply','1,2',current_timestamp()),
('pharmacy_claim','pay_type','1,2',current_timestamp());

In [0]:
%sql
INSERT INTO main.meta.dq_column_group
(table_name, column_name, merge_strategy, delimiter, active_flag)
VALUES
('enrollment', 'pay_type', 'concat', ',', 'Y');


In [0]:
%sql
INSERT INTO main.meta.enrollment
(
  patient_id,
  patient_gender,
  patient_year_of_birth,
  patient_zip3,
  patient_state,
  date_start,
  date_end,
  benefit_type,
  pay_type
)
VALUES
-- 1–5 : NULL CHECK failures
('E001', NULL, 1985, '123', 'CA', DATE '2020-01-01', DATE '2020-12-31', 'MED', 'COMM'),
('E002', 'M', NULL, '123', 'CA', DATE '2020-01-02', DATE '2020-12-31', 'MED', 'COMM'),
('E003', 'F', 1990, NULL, 'CA', DATE '2020-01-03', DATE '2020-12-31', 'MED', 'COMM'),
('E004', 'M', 1992, '123', NULL, DATE '2020-01-04', DATE '2020-12-31', 'MED', 'COMM'),
('E005', 'F', 1980, '123', 'CA', DATE '2020-01-05', NULL, 'MED', 'COMM'),

-- 6–10 : REGEX failures
('E006', 'X', 1985, '123', 'CA', DATE '2020-01-06', DATE '2020-12-31', 'MED', 'COMM'), -- gender
('E007', 'M', 1985, '12A', 'CA', DATE '2020-01-07', DATE '2020-12-31', 'MED', 'COMM'), -- zip
('E008', 'F', 1985, '1234', 'CA', DATE '2020-01-08', DATE '2020-12-31', 'MED', 'COMM'), -- zip
('E009', 'M', 1985, '123', 'California', DATE '2020-01-09', DATE '2020-12-31', 'MED', 'COMM'), -- state
('E010', 'F', 1985, '123', 'cA', DATE '2020-01-10', DATE '2020-12-31', 'MED', 'COMM'), -- state

-- 11–15 : RANGE / SCHEMA failures
('E011', 'M', 1800, '123', 'CA', DATE '2020-01-11', DATE '2020-12-31', 'MED', 'COMM'), -- DOB too low
('E012', 'F', 3000, '123', 'CA', DATE '2020-01-12', DATE '2020-12-31', 'MED', 'COMM'), -- DOB too high
('E013', 'M', -5, '123', 'CA', DATE '2020-01-13', DATE '2020-12-31', 'MED', 'COMM'), -- negative DOB
('E014', 'F', 9999, '123', 'CA', DATE '2020-01-14', DATE '2020-12-31', 'MED', 'COMM'), -- unrealistic DOB
('E015', 'M', 0, '123', 'CA', DATE '2020-01-15', DATE '2020-12-31', 'MED', 'COMM'),

-- 16–20 : DATE LOGIC failures (end < start)
('E016', 'F', 1985, '123', 'CA', DATE '2020-02-01', DATE '2020-01-31', 'MED', 'COMM'),
('E017', 'M', 1985, '123', 'CA', DATE '2020-03-01', DATE '2020-02-01', 'MED', 'COMM'),
('E018', 'F', 1985, '123', 'CA', DATE '2020-04-01', DATE '2020-03-01', 'MED', 'COMM'),
('E019', 'M', 1985, '123', 'CA', DATE '2020-05-01', DATE '2020-04-01', 'MED', 'COMM'),
('E020', 'F', 1985, '123', 'CA', DATE '2020-06-01', DATE '2020-05-01', 'MED', 'COMM');


In [0]:
%sql
INSERT INTO main.meta.pharmacy_claim
(
  claim_id,
  patient_id,
  date_service,
  ndc,
  fill_number,
  days_supply,
  dispensed_quantity,
  pay_type,
  copay_coinsurance,
  submitted_gross_due,
  paid_gross_due
)
VALUES
('R004','P004',try_cast('01-01-2024' AS DATE),'ABC123',1,30,30,'COMM',10,100,100),
('R005','P005',try_cast('20240105' AS DATE),'XYZ999',1,30,30,'COMM',10,100,100),
('R006','P006',try_cast('2024/01/06' AS DATE),'AAA111',1,30,30,'COMM',10,100,100),
('R007','P007',try_cast('2024-13-40' AS DATE),'ndc@12',1,30,30,'COMM',10,100,100);


In [0]:
%sql
INSERT INTO main.meta.pharmacy_claim
(
  claim_id,
  patient_id,
  date_service,
  ndc,
  fill_number,
  days_supply,
  dispensed_quantity,
  pay_type,
  copay_coinsurance,
  submitted_gross_due,
  paid_gross_due
)
VALUES
('RX01','P001','2024-01-01','abc123',1,30,30,'COMM',10,100,100),
('RX02','P002','2024-01-02','ABC-12',1,30,30,'COMM',10,100,100),
('RX03','P003','2024-01-03','AbC123',1,30,30,'COMM',10,100,100),
('RX04','P004','2024-01-04','NDC@99',1,30,30,'COMM',10,100,100);
